### Import libraries

In [3]:
# Import Libraries
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from dash.dependencies import Input, Output
import socket

### Customize

In [4]:
# Customize parameters
my_input_file='0402 my_input.csv'
my_x_fields = ['Revenue_Growth', 'Customer_Satisfaction', 'Service_Time', 'Training_Hours']

### Load data and run clustering 

In [5]:
# Load Dataset 
data = pd.read_csv(my_input_file)

# ML Clustering Analysis
X = data[my_x_fields]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(X_scaled)


C:\Users\ahmed\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


### Launch Dashboard

In [6]:
#Display Dash server address
ip_address = socket.gethostbyname(socket.gethostname())
print(f"Dash app running at http://{ip_address}:8050")

# Launch Dash App
app = Dash(__name__)

app.layout = html.Div([
    html.H3("Balanced Scorecard Dashboard with ML Insights"),

    dcc.Dropdown(
        id='metric',
        options=[
            {'label': 'Revenue Growth (%)', 'value': 'Revenue_Growth'},
            {'label': 'Customer Satisfaction', 'value': 'Customer_Satisfaction'},
            {'label': 'Service Time (min)', 'value': 'Service_Time'},
            {'label': 'Training Hours', 'value': 'Training_Hours'}
        ],
        value='Revenue_Growth',
        clearable=False
    ),

    dcc.Graph(id='bar-graph'),

    html.Br(),
    html.H2("Clustered Branch Profiles"),
    dcc.Graph(
        figure=px.scatter_3d(
            data,
            x='Revenue_Growth',
            y='Customer_Satisfaction',
            z='Training_Hours',
            color='Cluster',
            hover_name='Branch',
            title="ML-Based Branch Clustering"
        )
    )
])

@app.callback(
    Output('bar-graph', 'figure'),
    [Input('metric', 'value')]
)

def update_bar_chart(metric):
    fig = px.bar(data, x='Branch', y=metric, color='Cluster', barmode='group',
                 title=f"{metric} by Branch (Clustered)")
    return fig

if __name__ == '__main__':
   
    # For external access prefix the first option  with # and remove the # from the second option
    app.run(host='127.0.0.1', port=8050, debug=True)
    #app.run(host='0.0.0.0', port=8050, debug=True)


Dash app running at http://192.168.1.73:8050
